Importing libraries

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import cv2
import os
import glob

Model Definition

In [ ]:
from functools import partial

DefaultConv2D = partial(keras.layers.Conv2D,
                       kernel_size = 3, activation = 'relu', padding = 'same')

model = keras.models.Sequential([
    DefaultConv2D(filters = 32, input_shape = [100, 100, 1]),
    keras.layers.MaxPooling2D(pool_size=3),
    DefaultConv2D(filters= 64),
    keras.layers.MaxPooling2D(pool_size=3),
    keras.layers.Flatten(),
    keras.layers.Dense(units = '128', activation = 'relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(units = '2', activation = 'softmax'),
])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 100, 100, 32)      320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 33, 33, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 11, 11, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 7744)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               991360    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)              

Dataset import

In [ ]:
size = 144

train_data = []
train_labels = []

for per in os.listdir('/home/darthnoobisuke/Downloads/archive/sign_data/train/'):
    for data in glob.glob('/home/darthnoobisuke/Downloads/archive/sign_data/train/'+per+'/*.*'):
        img = cv2.imread(data,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (size, size))
        train_data.append([img])
        if per[-1]=='g':
            train_labels.append(np.array(1))
        else:
            train_labels.append(np.array(0))

train_data = np.array(train_data)/255.0
train_labels = np.array(train_labels)

test_data = []
test_labels = []

for per in os.listdir('/home/darthnoobisuke/Downloads/archive/sign_data/test/'):
    for data in glob.glob('/home/darthnoobisuke/Downloads/archive/sign_data/test/'+per+'/*.*'):
        img = cv2.imread(data,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (size, size))
        test_data.append([img])
        if per[-1]=='g':
            test_labels.append(1)
        else:
            test_labels.append(0)

test_data = np.array(test_data)/255.0
test_labels = np.array(test_labels)

In [ ]:
train_labels = tf.keras.utils.to_categorical(train_labels, dtype="float32")

In [ ]:
train_data = train_data.reshape(-1, size,size, 1)
test_data = test_data.reshape(-1, size,size, 1)

train_data,train_labels = shuffle(train_data,train_labels)
test_data,test_labels = shuffle(test_data,test_labels)

Compile and training the model

In [ ]:
%load_ext tensorboard
import datetime

opt = keras.optimizers.Adam(learning_rate = 0.0005)
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics = ["accuracy"])

log_dir = "logs/fit/Sign_Check" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=6, min_lr=0.001)
    
early_stopping_cb = keras.callbacks.EarlyStopping(patience=9,
                                                  restore_best_weights=True)

checkpoint_cb = keras.callbacks.ModelCheckpoint("Sign_Check.h5",save_best_only=True)

history = model.fit(train_data, train_labels,
                    epochs = 60,
                    batch_size = 64,
                    validation_split=.3, 
                    callbacks = [reduce_lr, early_stopping_cb, tensorboard_callback, checkpoint_cb])

Epoch 1/40
 1/19 [>.............................] - ETA: 0s - loss: 0.7055 - accuracy: 0.4375WARNING:tensorflow:From /home/darthnoobisuke/ml/env/lib/python3.6/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
19/19 [==============================] - 3s 139ms/step - loss: 0.7067 - accuracy: 0.5277 - val_loss: 0.6873 - val_accuracy: 0.5414
Epoch 2/40
19/19 [==============================] - 3s 138ms/step - loss: 0.6804 - accuracy: 0.5815 - val_loss: 0.6679 - val_accuracy: 0.6889
Epoch 3/40
19/19 [==============================] - 3s 133ms/step - loss: 0.6408 - accuracy: 0.6612 - val_loss: 0.6525 - val_accuracy: 0.5758
Epoch 4/40
19/19 [==============================] - 3s 136ms/step - loss: 0.5812 - accuracy: 0.6898 - val_loss: 0.4954 - val_accuracy: 0.8040
Epoch 5/40
19/19 [==============================] - 3s 

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
pred = model.predict(test_data)
predict =np.argmax(pred,axis=1)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(predict , test_labels)

0.98

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

In [ ]:
f1_score(predict, test_labels)

0.9794238683127572

In [ ]:
recall_score(predict, test_labels)

1.0

In [ ]:
precision_score(predict,test_labels)

0.9596774193548387

In [ ]:
confusion_matrix(predict,test_labels)

array([[252,  10],
       [  0, 238]])